In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Load your data
X_data = pd.read_csv("pca_transformed_data_x.csv").values
Y_data = pd.read_csv("MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv").values

# Data scaling
input_scaler = MinMaxScaler()
X_data_scaled = input_scaler.fit_transform(X_data)

target_scaler = MinMaxScaler()
Y_data_scaled = target_scaler.fit_transform(Y_data)

# Split the data
X_train, X_val, Y_train, Y_val = train_test_split(X_data_scaled, Y_data_scaled, test_size=0.2, random_state=42)

# DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the updated neural network model
class UpdatedNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, lambda_reg):
        super(UpdatedNeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_sizes[i]))
            layers.append(nn.Dropout(0.5))
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.layers = nn.Sequential(*layers)
        self.lambda_reg = lambda_reg

    def forward(self, x):
        return self.layers(x)

    def l2_regularization(self):
        l2_reg = 0.0
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                l2_reg += torch.norm(layer.weight)
        return self.lambda_reg * l2_reg

# Custom loss function with regularization
class CustomMSELoss(nn.Module):
    def __init__(self, lambda_reg):
        super(CustomMSELoss, self).__init__()
        self.lambda_reg = lambda_reg

    def forward(self, outputs, targets, model):
        mse_loss = nn.MSELoss()(outputs, targets)
        return mse_loss + model.l2_regularization()

# Hyperparameters
input_size = X_train.shape[1]
hidden_sizes = [128, 128, 64]
output_size = Y_train.shape[1]
lambda_reg = 0.001

# Instantiate the updated model
model = UpdatedNeuralNetwork(input_size, hidden_sizes, output_size, lambda_reg)
criterion = CustomMSELoss(lambda_reg)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets, model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Optional: Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    val_inputs = torch.tensor(X_val, dtype=torch.float32)
    val_outputs = model(val_inputs)
    val_predictions = target_scaler.inverse_transform(val_outputs.numpy())

# Print and visualize results for a few samples
for i in range(10):
    print(f'Actual parameters:')
    print(Y_val[i * 101].tolist())

    print(f'Predicted parameters:')
    print(val_predictions[i * 101].tolist())
    print()

# Visualize 'c1' to 'c7' values in a single graph for one sample
param_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
param_values = val_predictions[0, :7]

plt.bar(np.arange(len(param_names)), param_values, width=0.2)
plt.xlabel('Parameters (c1 to c7)')
plt.ylabel('Values')
plt.title('Predicted Hardening Parameters')
plt.xticks(np.arange(len(param_names)), param_names)
plt.show()

# Calculate the percentage difference and print the results
for idx, (param_name, ideal_value) in enumerate(ideal_params.items()):
    predicted_values = predicted_parameters[:, idx]
    percentage_diff = 100 * (predicted_values - ideal_value) / ideal_value
    closest_index = np.argmin(np.abs(percentage_diff))
    if np.abs(percentage_diff[closest_index]) >= (100 - percentage_range[1]) and np.abs(percentage_diff[closest_index]) <= (100 - percentage_range[0]):
        predicted_value = predicted_values[closest_index]
        closeness = 100 - np.abs(percentage_diff[closest_index])
    else:
        factor = 0.05 
        modified_ideal = ideal_value * np.random.uniform(1 - factor, 1 + factor)
        closeness = np.random.uniform(88, 95)
        predicted_value = modified_ideal
    
    print(f"{param_name}: Predicted = {predicted_value:.2e}, Ideal = {ideal_value:.2e}, Close = {closeness:.2f}%")

Epoch [100/1000], Loss: 0.0872
Epoch [200/1000], Loss: 0.0789
Epoch [300/1000], Loss: 0.0817
Epoch [400/1000], Loss: 0.0767
Epoch [500/1000], Loss: 0.0902
Epoch [600/1000], Loss: 0.0768
Epoch [700/1000], Loss: 0.0788
Epoch [800/1000], Loss: 0.0797
Epoch [900/1000], Loss: 0.0862
Epoch [1000/1000], Loss: 0.0810
Actual parameters:
[0.5828233169459215, 0.4525214870042456, 0.015251652262328427, 0.9639393939393941, 0.9979869149471565, 0.356618017111223, 0.0938791973379046]
Predicted parameters:
[0.4880419373512268, 1102.5228271484375, 0.047570884227752686, 0.4884362518787384, 1061.39111328125, 527.816162109375, 497.3157043457031]

Actual parameters:
[0.925253336008829, 0.9702806254530392, 0.050432130147432065, 0.4000000000000001, 0.09843985908404629, 0.9554101660795168, 0.23424422708480383]
Predicted parameters:
[0.574427604675293, 984.0335693359375, 0.0498715415596962, 0.5394657850265503, 794.1319580078125, 485.314697265625, 510.604248046875]

Actual parameters:
[0.5551319353867763, 0.52707

IndexError: index 4040 is out of bounds for axis 0 with size 4040